In [1]:
import json, jmespath
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
nltk.download('punkt')

# url for reference: https://www.wsav.com/news/national-news/student-suspended-after-posting-warnings-of-rapist/

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/felipemarineli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
path_to_json = "/Users/felipemarineli/Environments/NLP_clutter/sample2.json"
f = "body.content.text"

with open(path_to_json, 'rb') as json_sample:
    parsed_json = json.load(json_sample)

expression = jmespath.compile(f)

text = expression.search(parsed_json)

print(text)

CAPE ELIZABETH, Maine (WCSH) – A sophomore at Maine’s Cape Elizabeth High School was suspended for bullying after she left sticky notes in two girls bathrooms reading, “There’s a rapist in our school and you know who it is.” “It makes me angry that I’m being punished for bullying and a rapist isn’t being punished for raping people,” Aela Mansmann said Friday. “I felt this was important — that this was common knowledge.” Mansmann said allegations of sexual assaults have occurred for years and the administration hasn’t listened to the allegations. She said she posted the notes because she and other students don’t feel safe at school. “On a day-to-day level we don’t feel believed,” she said. “We don’t feel supported.” School officials say the law requires they are required by law to investigate any time a student bullies another student, and said they are confident in their investigation of the incident. Read more: http://bit.ly/2AYfgGE Related Content Police: High school coach lifted $40

In [3]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

tokens = sent_detector.tokenize(text.strip())

print(tokens)

['CAPE ELIZABETH, Maine (WCSH) – A sophomore at Maine’s Cape Elizabeth High School was suspended for bullying after she left sticky notes in two girls bathrooms reading, “There’s a rapist in our school and you know who it is.” “It makes me angry that I’m being punished for bullying and a rapist isn’t being punished for raping people,” Aela Mansmann said Friday.', '“I felt this was important — that this was common knowledge.” Mansmann said allegations of sexual assaults have occurred for years and the administration hasn’t listened to the allegations.', 'She said she posted the notes because she and other students don’t feel safe at school.', '“On a day-to-day level we don’t feel believed,” she said.', '“We don’t feel supported.” School officials say the law requires they are required by law to investigate any time a student bullies another student, and said they are confident in their investigation of the incident.', 'Read more: http://bit.ly/2AYfgGE Related Content Police: High school

In [4]:
df = pd.DataFrame(tokens)
df.columns = ['text_chunk']
df['target'] = ''

df.head(6)

,text_chunk,target
0,"CAPE ELIZABETH, Maine (WCSH) – A sophomore at ...",
1,“I felt this was important — that this was com...,
2,She said she posted the notes because she and ...,
3,"“On a day-to-day level we don’t feel believed,...",
4,“We don’t feel supported.” School officials sa...,
5,Read more: http://bit.ly/2AYfgGE Related Conte...,


In [5]:
#df.to_csv("test.csv")

# Annotation takes place

In [6]:
df = pd.read_csv("/Users/felipemarineli/Environments/NLP_clutter/test.csv", index_col=0)

In [7]:
df.head(6)

,text_chunk,target
0,"CAPE ELIZABETH, Maine (WCSH) – A sophomore at ...",0
1,“I felt this was important — that this was com...,0
2,She said she posted the notes because she and ...,0
3,"“On a day-to-day level we don’t feel believed,...",0
4,“We don’t feel supported.” School officials sa...,0
5,Read more: http://bit.ly/2AYfgGE Related Conte...,1


In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text_chunk'])
word_index = tokenizer.word_index

for i, chunk in enumerate(df['text_chunk']):
    df['text_chunk'][i] = tokenizer.texts_to_sequences([chunk])[0]

/Users/felipemarineli/Environments/NLP_clutter/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df.head(6)

,text_chunk,target
0,"[15, 16, 33, 34, 35, 2, 36, 17, 37, 15, 16, 18...",0
1,"[63, 64, 26, 9, 65, 66, 22, 26, 9, 67, 68, 5, ...",0
2,"[3, 6, 3, 77, 7, 20, 78, 3, 4, 79, 80, 12, 13,...",0
3,"[82, 2, 29, 11, 29, 83, 84, 12, 13, 85, 5, 3, 6]",0
4,"[86, 12, 13, 87, 5, 1, 88, 89, 7, 30, 90, 31, ...",0
5,"[102, 103, 104, 105, 106, 107, 108, 109, 110, ...",1


In [10]:
df.dtypes

text_chunk    object
target         int64
dtype: object

In [11]:
data = df['text_chunk'].apply(pd.Series)
data.shape

(6, 64)

In [12]:
data.dtypes

0     float64
1     float64
2     float64
3     float64
4     float64
       ...   
59    float64
60    float64
61    float64
62    float64
63    float64
Length: 64, dtype: object

In [13]:
targets = df.pop('target')
targets.shape

(6,)

In [14]:
targets.dtypes

dtype('int64')

In [15]:
dataset = tf.data.Dataset.from_tensor_slices((data.values, targets.values))

In [16]:
dataset

<TensorSliceDataset shapes: ((64,), ()), types: (tf.float64, tf.int64)>

In [17]:
for feat, targ in dataset.take(6):
    print('Features: {}, Target: {}'.format(feat, targ))

Features: [15. 16. 33. 34. 35.  2. 36. 17. 37. 15. 16. 18.  1.  9. 38.  8. 19. 39.
  3. 40. 41. 20. 10. 42. 43. 44. 45. 46.  2. 21. 10. 47.  1.  4. 48. 49.
 50. 51. 52.  5. 53. 54. 55. 56. 22. 57. 23. 24.  8. 19.  4.  2. 21. 58.
 23. 24.  8. 59. 60.  5. 61. 25.  6. 62.], Target: 0
Features: [63. 64. 26.  9. 65. 66. 22. 26.  9. 67. 68.  5. 25.  6. 27. 28. 69. 70.
 71. 72.  8. 73.  4.  7. 74. 75. 76. 11.  7. 27. nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan], Target: 0
Features: [ 3.  6.  3. 77.  7. 20. 78.  3.  4. 79. 80. 12. 13. 81. 17.  1. nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan], Target: 0
Features: [82.  2. 29. 11. 29. 83. 84. 12. 13. 85.  5.  3.  6. nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan na

In [18]:
train_dataset = dataset.shuffle(len(df)).batch(1)

In [19]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
  ])

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

In [20]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Train for 6 steps
Epoch 1/15
6/6 [==============================] - 0s 79ms/step - loss: nan - accuracy: 0.8333
Epoch 2/15
6/6 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.8333
Epoch 3/15
6/6 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.8333
Epoch 4/15
6/6 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8333
Epoch 5/15
6/6 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.8333
Epoch 6/15
6/6 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.8333
Epoch 7/15
6/6 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.8333
Epoch 8/15
6/6 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.8333
Epoch 9/15
6/6 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.8333
Epoch 10/15
6/6 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.8333
Epoch 11/15
6/6 [==============================] 

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  650       
_________________________________________________________________
dense_1 (Dense)              multiple                  110       
_________________________________________________________________
dense_2 (Dense)              multiple                  11        
Total params: 771
Trainable params: 771
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.save('/Users/felipemarineli/Library/Mobile Documents/com~apple~CloudDocs/Models/clutter.h5')